In [3]:
import pandas as pd
import numpy as np

import heapq

In [4]:
with open('../../inputs/0081_matrix.txt', 'r') as f:
    matrix_string = f.read()

This is a classic dynamic programming problem where we calculate the minimum path sum.

It's similar to the maximum path sum (problem 18 and 67) in that from each node, we can only go in two directions, so we can use a dynamic programming approach similar to that problem, except the shape here is a rhombus (imagine turning the matrix $45^{\circ}$), instead of a triangle. In the triangle version, we guarantee that the next row has more values than the previous row (so each number in the previous row has two numbers below it that we can check); in the rhombus, we have to be more careful about boundaries, since numbers on the edge can only go to 1 number in the next row.

It's also similar to the lattice paths (problem 15), since we can try to analyze each path, in that we can only move right and down, and we can analyze each path similarly. However, the big difference is there we could speed up the calculation signifcantly by doing a mathematical (combinatoric) formulation, since we were just counting; in this problem, we need to actually take the sume, so we can't just use a general mathematical approach.

The first approach I use here is a greedy approach (bottom-up dynamic programming), more similar to the lattice paths. We precalculate the sum of the first column and first row, since they are only affected by one direction (affected by above for first column, affected by left for first row). Then at each other number (starting from $(i,j) = (1,1)$), we add on the minimum of the number above and the number to the left. In notation, for a matrix $A$, we assign $A_{i,j} = A_{i,j} + \min(A_{i,j-1}, A_{i-1,j})$.

In [5]:
# read in matrix
matrix = np.array([[int(n) for n in row.split(',')] for row in matrix_string.split('\n')], int)
m = len(matrix)
n = len(matrix[0])

# sum first column, since path must come from above
for i in range(1,m):
    matrix[i, 0] += matrix[i-1, 0]

# sum first row, since path must come from left
for j in range(1,n):
    matrix[0, j] += matrix[0, j-1]

# at each other row and col, greedily add on the minimum of the number above and number to the left
for i in range(1,m):
    for j in range(1,n):
        matrix[i,j] += min(matrix[i,j-1], matrix[i-1, j])

# the bottom right corner stores the path sum
print(matrix[-1,-1])

427337


The second approach here is more like the triangular maximum path sum, where we start at the bottom right corner and move left and up back toward the start node.

In [6]:
# read in matrix
matrix = np.array([[int(n) for n in row.split(',')] for row in matrix_string.split('\n')], int)
m = len(matrix)
n = len(matrix[0])

for i in range(m)[::-1]:
    for j in range(n)[::-1]:
        # if in bottom right corner, you don't need to do anything
        if i == m-1 and j == n-1:
            continue

        # you're on last row, so you only need to look right
        if i == m-1:
            matrix[i, j] += matrix[i,j+1]
            continue

        # you're on the last column so you only need to look below
        if j == n-1:
            matrix[i,j] += matrix[i+1,j]
            continue
        
        # look right and down and see which path would give a minimum
        matrix[i,j] += min(matrix[i+1,j], matrix[i,j+1])

# the top left corner stores the path sum
print(matrix[0,0])

427337


Another approach would be the [BFS (breadth-first search)][1] algorithm, which uses a queue and sums along the way.

  [1]: https://en.wikipedia.org/wiki/Breadth-first_search

In [9]:
# read in the matrix
matrix = np.array([[int(n) for n in row.split(',')] for row in matrix_string.split('\n')], int)
m = len(matrix)
n = len(matrix[0])
sum_list = []

r, c = 0, 0
ssf = 0
visited = np.zeros_like(matrix)

# use a queue with elements (row, col, sum_so_far)
queue = [(ssf, r, c)]

# transform into a minheap priority queue
heapq.heapify(queue)
while queue:
    ssf, r, c = heapq.heappop(queue)
    # if out of bounds then skip
    # if you have already visited earlier, there must have been a shorter path
    # to this number before, so skip
    if r > m-1 or c > n-1 or visited[r,c]:
        continue
    if r == m-1 and c == n-1:
        sum_list.append(ssf + matrix[r,c])
        continue
    
    visited[r,c] = 1

    # append right, up, and down moves
    heapq.heappush(queue, (ssf + matrix[r,c], r, c+1))
    heapq.heappush(queue, (ssf + matrix[r,c], r+1, c))

print(np.min(sum_list))

427337
